# 11. Hafta Lab-1: Parola Oluşturma
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
## İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
### Dr. Ferhat Özgür Çatak
Bu lab çalışması kapsamında rockyou.txt veri kümesi kullanılarak parolalar oluşturulacaktır.

In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import io
import random

Using TensorFlow backend.


### Veri kümesi
rockyou.txt veri dosyasından ilk 50.000 karakter oku

In [2]:
filename = "rockyou_simple.txt"
#raw_text = open(filename).read()

with io.open(filename,'r',encoding='utf8') as f:
    raw_text = f.read()

raw_text = raw_text.replace("\r", " ").replace("\n"," ")[0:50000]

### Sözlük oluşturulması
Her bir karakter kullanılacak şekilde sözlük oluştur

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
" ".join(chars)

'  ! " # $ % & \' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > @ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z [ \\ ] ^ _ ` a b c d e f g h i j k l m n o p q r s t u v w x y z ~ ¬ ° ± µ Ñ ü π ‡ ∏ ∑ √ ∞'

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  50000
Total Vocab:  103


### Veri kümesinin hazırlanması
integer olarak kodlanmış input-output çiftlerini oluştur.

In [5]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(len(dataX), len(dataY))
print(dataX[0])
print(len(dataY))

49900 49900
[73, 81, 78, 23, 18, 20, 0, 76, 66, 81, 86, 64, 74, 64, 75, 64, 0, 44, 68, 87, 72, 66, 78, 3, 17, 0, 36, 49, 40, 51, 52, 19, 18, 0, 17, 17, 21, 21, 19, 18, 17, 21, 0, 20, 18, 17, 25, 18, 21, 24, 0, 76, 78, 77, 64, 67, 64, 82, 0, 73, 17, 17, 18, 25, 24, 21, 0, 88, 68, 75, 75, 78, 86, 25, 10, 0, 67, 78, 70, 66, 71, 78, 86, 22, 20, 0, 73, 66, 73, 66, 0, 66, 73, 66, 22, 24, 20, 19, 25, 0]
49900


In [6]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
X.shape

(49900, 100, 1)

### Model

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), activation='tanh', return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(128, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Model Yukle
Daha önce eğitilmiş olan modeli yükle

In [9]:

filename = "01-rockyou.hdf5"
model.load_weights(filename)


In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# define the checkpoint
checkpoint = ModelCheckpoint(filename, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
# model.fit(X, y, epochs=10, batch_size=5000, callbacks=callbacks_list, verbose=1)

### Pasword Generation
10 kere parola oluştur. Parola oluşturulurken metin içinde parolalar alınarak oluşturulacaktır.

In [11]:
for i in range(20):
  
    space_idx = random.choice([pos for pos, char in enumerate(raw_text) if char == " "])
    pattern = dataX[space_idx]
    
    print( "Seed:")
    print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")
    
    # generate characters
    sys.stdout.write("-")
    for i in range(100):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        '''
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        '''
        index = model.predict_classes(x, verbose=0)[0]
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    sys.stdout.write("-\n")

Seed:
"  Younes07 ccavallero1 jkljkljkljkl 0845764309 21wqsaxz90 kamkar2 guaguasita123 KHASI houndgrey 22213 "
-13 3443myslides achi1404 Icanskate13 3889223ROCIO 028700490a missy259 jabene flgfus tuoke54 w005894 -
Seed:
"  aceofdan 1160100168008 corchnocs sikilbau Bammie10 ginzice faded99 delil@h1 xikit4TEAMo mor1101 bhe "
-bheron bigbes KOLUO6383 eeil2001 sentned loomyen1 0167855442 meme15128 jerii 0074744976 0865356756 m-
Seed:
"  kaan123. 16deoctubre orrrx 497808 MATT4444 017951573 arsenalpow wajacanla xher0tic* sureno13m petch "
-muangcaan grokyyonen marieru geumeeter ealllira1 018058804 0818697382 lantessa tatssoner1 082282764 -
Seed:
"  tshbr1328 093578948 mywaykarena lifeboss lynx1212 CMONEY4 snj122503 0855503261 michlel13 bobby2007  "
-088064158 asiakforero yonaboo92 Nhatcolle tasytan jracdte papzanu tipe5402 cicdatcypo slovaduclu 085-
Seed:
"  whyrich cloee. murdocs870210 936195 TOOTYMAX squamos torie catakin 074348706 mg12893 biggrils lucia "
-nto 0831107 0829110746 ilsssod13